In [19]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import seaborn as sns
from scipy import stats
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from pycaret.regression import *

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

In [20]:
'''train_a = pd.read_parquet('A/train_targets.parquet')
train_b = pd.read_parquet('B/train_targets.parquet')
train_c = pd.read_parquet('C/train_targets.parquet')

X_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')
X_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')
X_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')

X_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')
X_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')
X_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')

X_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')
X_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')
X_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')'''

"train_a = pd.read_parquet('A/train_targets.parquet')\ntrain_b = pd.read_parquet('B/train_targets.parquet')\ntrain_c = pd.read_parquet('C/train_targets.parquet')\n\nX_train_estimated_a = pd.read_parquet('A/X_train_estimated.parquet')\nX_train_estimated_b = pd.read_parquet('B/X_train_estimated.parquet')\nX_train_estimated_c = pd.read_parquet('C/X_train_estimated.parquet')\n\nX_train_observed_a = pd.read_parquet('A/X_train_observed.parquet')\nX_train_observed_b = pd.read_parquet('B/X_train_observed.parquet')\nX_train_observed_c = pd.read_parquet('C/X_train_observed.parquet')\n\nX_test_estimated_a = pd.read_parquet('A/X_test_estimated.parquet')\nX_test_estimated_b = pd.read_parquet('B/X_test_estimated.parquet')\nX_test_estimated_c = pd.read_parquet('C/X_test_estimated.parquet')"

In [21]:
# PREPROCESSING
def preprocessing(targets, observed, estimated, test):
   
    # Ensure the datetime columns are in datetime format
    targets['time'] = pd.to_datetime(targets['time'])
    observed['date_forecast'] = pd.to_datetime(observed['date_forecast'])
    estimated['date_forecast'] = pd.to_datetime(estimated['date_forecast'])
    test['date_forecast'] = pd.to_datetime(test['date_forecast'])

    # Resample observed, estimated, and test data to 1 hour using mean() as aggregator
    # and drop rows where all columns are NaN
    observed_resampled = observed.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
    estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
    test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
    
    # Merge the observed and estimated data
    weather_data = pd.concat([observed_resampled, estimated_resampled])

    # Merge with target values
    merged_data = pd.merge(targets, weather_data, how='inner', left_on='time', right_on='date_forecast')

    # Drop non-feature columns
    merged_data = merged_data.drop(columns=['time', 'date_forecast', 'pv_measurement'])
    
    return merged_data, test_resampled

In [22]:
from pycaret.regression import *
def process_location(X, y, location_name):
    # Combine feature data and target into a single DataFrame
    data = X.copy()
    data['target'] = y

    # Setup the environment in PyCaret
    exp_reg = setup(data=data, target='target', session_id=123,
                    categorical_features=['dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx'],
                    html=False,
                    #silent=True,  # It's important to set silent=True to avoid PyCaret's interactive input requests
                    experiment_name=f'exp_{location_name}')

    # Create a LightGBM model
    lightgbm = create_model('lightgbm')

    # Tune the model
    tuned_lightgbm = tune_model(lightgbm)

    # Create a bagged version of the tuned model
    bagged_lightgbm = ensemble_model(tuned_lightgbm, method='Bagging')

    # Finalize the model - this will train it on the complete dataset
    final_model = finalize_model(bagged_lightgbm)

    # Save the model for future use
    save_model(final_model, f'final_model_for_location_{location_name}')

    return final_model

In [23]:
'''from pycaret.time_series import *
def process_location_time(X, y, location_name):
    data = X.copy()
    data['target'] = y
    data.dropna(axis=1,inplace=True)
    exp_reg = setup(data=data, target='target', session_id=123,
                    #categorical_features=['dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx'],
                    html=False,
                    #silent=True,  # It's important to set silent=True to avoid PyCaret's interactive input requests
    experiment_name=f'exp_{location_name}')
    best = compare_models(include=['dt_cds_dt', 'rf_cds_dt','auto_arima' ])
    tuned_best = tune_model(best)
    #bagged_best = ensemble_model(tuned_best, method='Bagging')
    final_model = finalize_model(tuned_best)
    save_model(final_model, f'final_model_for_location_{location_name}')
    return final_model'''

"from pycaret.time_series import *\ndef process_location_time(X, y, location_name):\n    data = X.copy()\n    data['target'] = y\n    data.dropna(axis=1,inplace=True)\n    exp_reg = setup(data=data, target='target', session_id=123,\n                    #categorical_features=['dew_or_rime:idx', 'is_day:idx', 'is_in_shadow:idx'],\n                    html=False,\n                    #silent=True,  # It's important to set silent=True to avoid PyCaret's interactive input requests\n    experiment_name=f'exp_{location_name}')\n    best = compare_models(include=['dt_cds_dt', 'rf_cds_dt','auto_arima' ])\n    tuned_best = tune_model(best)\n    #bagged_best = ensemble_model(tuned_best, method='Bagging')\n    final_model = finalize_model(tuned_best)\n    save_model(final_model, f'final_model_for_location_{location_name}')\n    return final_model"

In [24]:
'''locations = ['A', 'B', 'C']
all_models = []

for loc in locations:
    # Load your data
    train = pd.read_parquet(f'{loc}/train_targets.parquet').fillna(0)
    X_train_estimated = pd.read_parquet(f'{loc}/X_train_estimated.parquet')
    X_train_observed = pd.read_parquet(f'{loc}/X_train_observed.parquet')
    X_test_estimated = pd.read_parquet(f'{loc}/X_test_estimated.parquet')

    # Preprocess data
    X_train, X_test = preprocessing(train, X_train_observed, X_train_estimated, X_test_estimated)
    y_train = train['pv_measurement'].values

    # Ensure X and y have the same length
    min_length = min(len(X_train), len(y_train))
    X_train, y_train = X_train.iloc[:min_length], y_train[:min_length]

    # Process the location data and get the final model
    final_model = process_location(X_train, y_train, loc)

    # Store the model in all_models list
    all_models.append(final_model)'''

"locations = ['A', 'B', 'C']\nall_models = []\n\nfor loc in locations:\n    # Load your data\n    train = pd.read_parquet(f'{loc}/train_targets.parquet').fillna(0)\n    X_train_estimated = pd.read_parquet(f'{loc}/X_train_estimated.parquet')\n    X_train_observed = pd.read_parquet(f'{loc}/X_train_observed.parquet')\n    X_test_estimated = pd.read_parquet(f'{loc}/X_test_estimated.parquet')\n\n    # Preprocess data\n    X_train, X_test = preprocessing(train, X_train_observed, X_train_estimated, X_test_estimated)\n    y_train = train['pv_measurement'].values\n\n    # Ensure X and y have the same length\n    min_length = min(len(X_train), len(y_train))\n    X_train, y_train = X_train.iloc[:min_length], y_train[:min_length]\n\n    # Process the location data and get the final model\n    final_model = process_location(X_train, y_train, loc)\n\n    # Store the model in all_models list\n    all_models.append(final_model)"

In [25]:
locations = ['A', 'B', 'C']
all_predictions = []
final_df_list = [] 


for loc in locations:
    # Load your data
    train = pd.read_parquet(f'{loc}/train_targets.parquet').fillna(0)
    X_train_estimated = pd.read_parquet(f'{loc}/X_train_estimated.parquet')
    X_train_observed = pd.read_parquet(f'{loc}/X_train_observed.parquet')
    X_test_estimated = pd.read_parquet(f'{loc}/X_test_estimated.parquet')

    # Preprocess data
    X_train, X_test = preprocessing(train, X_train_observed, X_train_estimated, X_test_estimated)
    y_train = train['pv_measurement'].values

    # Ensure X and y have the same length
    min_length = min(len(X_train), len(y_train))
    X_train, y_train = X_train.iloc[:min_length], y_train[:min_length]

    # Process the location data and get the final model
    final_model = process_location(X_train, y_train, loc)
    X_test.dropna(axis=1,inplace=True)
    # Predict on new data
    predictions = predict_model(final_model, fh=len(X_test), X=X_test)

    print(predictions.columns)

    # Store predictions
    all_predictions.append(predictions['y_pred'])  

    # Prepare dataframe for visualization
    num_predictions = len(predictions)
    timestamps = pd.date_range(start='2023-05-01 00:00:00', periods=num_predictions, freq='H')

    df = pd.DataFrame({
        'time': timestamps,
        'prediction': predictions['y_pred'],
        'location': loc
    })

    final_df_list.append(df)

# Concatenate all the individual data frames and prepare for plotting
final_df = pd.concat(final_df_list, ignore_index=True)
final_df['time'] = pd.to_datetime(final_df['time'])
final_df.sort_values('time', inplace=True)

# Visualization
for loc in locations:
    # Filter data for each location
    temp_df = final_df[final_df['location'] == loc]
    
    plt.figure(figsize=(12, 6))  # Create a new figure for each location
    plt.plot(temp_df['time'], temp_df['prediction'], label=f'Location {loc}')
    plt.xlabel('Time')
    plt.ylabel('Predictions')
    plt.title(f'Predictions Over Time for Location {loc}')
    plt.legend()
    plt.show()

# If you need to save the final dataframe without 'time' and 'location' columns, you can do so before plotting
final_df_save = final_df.copy()
final_df_save = final_df_save.drop(columns=['time', 'location'])

                    Description        Value
0                    Session id          123
1                        Target       target
2                   Target type   Regression
3           Original data shape  (34061, 46)
4        Transformed data shape  (34061, 62)
5   Transformed train set shape  (23842, 62)
6    Transformed test set shape  (10219, 62)
7              Numeric features           42
8          Categorical features            3
9      Rows with missing values        97.4%
10                   Preprocess         True
11              Imputation type       simple
12           Numeric imputation         mean
13       Categorical imputation         mode
14     Maximum one-hot encoding           25
15              Encoding method         None
16               Fold Generator        KFold
17                  Fold Number           10
18                     CPU Jobs           -1
19                      Use GPU        False
20               Log Experiment        False
21        

           MAE          MSE      RMSE      R2   RMSLE    MAPE
Fold                                                         
0     202.0913  182170.5580  426.8144  0.8672  1.2504  1.3275
1     192.8785  175765.0472  419.2434  0.8669  1.2242  1.2884
2     188.3806  163246.3219  404.0375  0.8769  1.2707  1.5085
3     187.7374  162585.8416  403.2193  0.8873  1.2525  1.4801
4     199.6796  193202.7568  439.5484  0.8585  1.3544  1.8367
5     203.2024  210479.8879  458.7809  0.8444  1.3148  1.5133
6     197.9293  201171.7252  448.5217  0.8550  1.3122  1.7340
7     192.8957  169295.8879  411.4558  0.8771  1.1906  1.2148
8     199.7484  180669.0362  425.0518  0.8698  1.2225  1.2588
9     199.9936  210477.7095  458.7785  0.8466  1.3494  1.2449
Mean  196.4537  184906.4772  429.5452  0.8650  1.2742  1.4407
Std     5.2904   17208.5569   19.9353  0.0131  0.0533  0.2032


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


           MAE          MSE      RMSE      R2   RMSLE    MAPE
Fold                                                         
0     203.0405  186466.8836  431.8181  0.8641  1.3173  1.7052
1     195.4510  178940.7393  423.0139  0.8645  1.3483  1.2889
2     188.6688  168772.1163  410.8188  0.8727  1.3262  1.8837
3     184.3176  154213.0842  392.6997  0.8931  1.2919  1.9547
4     197.7693  192621.7028  438.8869  0.8589  1.3804  2.1457
5     200.4196  201603.8951  449.0032  0.8510  1.4109  1.6544
6     190.5849  184500.3472  429.5350  0.8670  1.3475  1.7576
7     198.1652  178601.7667  422.6130  0.8703  1.3573  1.3687
8     203.4899  183101.1893  427.9032  0.8681  1.3139  1.3808
9     201.9291  209483.1905  457.6933  0.8474  1.3560  1.3943
Mean  196.3836  183830.4915  428.3985  0.8657  1.3450  1.6534
Std     6.2173   14880.7463   17.4697  0.0120  0.0329  0.2748


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
'''sample_submission = pd.read_csv('sample_submission.csv')
sample_submission
sample_submission = sample_submission[['id']].merge(final_df[['id', 'prediction']], on='id', how='left')
sample_submission.to_csv('my_first_submission.csv', index=False)'''

final_predictions = np.concatenate(all_predictions)

# Save the final_predictions to CSV
df = pd.DataFrame(final_predictions, columns=['prediction'])
df['id'] = df.index
df = df[['id', 'prediction']]
df.to_csv('final_predictions.csv', index=False)